In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils

In [2]:
# Define directories
image_directory = '../data/deigo/annotate-frames'  # Original Images
depth_map_root_directory = '../data/deigo/depth-maps/'  # Depth Map Images
json_file = '../data/deigo/diego-pipe-annotations/annotations/instances_default.json'  # JSON file

In [3]:
# Paths to save results
bream_csv_path = '../data/deigo/results_bream_oyster.csv'
puffer_csv_path = '../data/deigo/results_puffer_oyster.csv'
scat_csv_path = '../data/deigo/results_scat_oyster.csv'

In [4]:
# Mapping of images to their specific depth map directories
depth_map_directories = {
    'bream_oyster_frame_000001.png': os.path.join(depth_map_root_directory, 'bream-oyster'),
    'puffer_oyster_frame_000001.png': os.path.join(depth_map_root_directory, 'puffer-at-night-oyster'),
    'scat_oyster_frame_000001.png': os.path.join(depth_map_root_directory, 'scat-oyster-reef-day')
}

In [5]:
# Load the COCO annotations
coco = COCO(json_file)

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '../data/deigo/diego-pipe-annotations/annotations/instances_default.json'

In [ ]:
# Create a mapping from category_id to category_name
cats = coco.loadCats(coco.getCatIds())
category_mapping = {cat['id']: cat['name'] for cat in cats}
print("Category mapping:")
print(category_mapping)

In [ ]:
def load_and_normalize_depth_map(depth_map_path):
    # Load the depth map image
    depth_map = cv2.imread(depth_map_path, cv2.IMREAD_UNCHANGED)
    if depth_map is None:
        print(f"Failed to load depth map at {depth_map_path}.")
        return None

    # Normalize the depth map based on its data type
    if depth_map.dtype == np.uint8:
        # 8-bit depth map
        depth_map_normalized = depth_map / 255.0
    else:
        # If depth map is already in float format between 0 and 1
        depth_map_normalized = depth_map

    return depth_map_normalized

In [ ]:
def create_mask_from_annotation(ann, img_height, img_width):
    segmentation = ann['segmentation']

    if isinstance(segmentation, list):  # Polygon format
        mask = np.zeros((img_height, img_width), dtype=np.uint8)
        # Convert polygons to a mask
        for poly in segmentation:
            poly = np.array(poly).reshape((-1, 2))  # Reshape to (N, 2)
            cv2.fillPoly(mask, [np.int32(poly)], 1)  # Fill the polygon with 1's

    elif isinstance(segmentation, dict) and 'counts' in segmentation:  # RLE format
        rle = segmentation
        if isinstance(rle['counts'], list):  # Uncompressed RLE
            rle = maskUtils.frPyObjects([rle], img_height, img_width)
        mask = maskUtils.decode(rle)  # Decode RLE to binary mask

    else:
        mask = None  # Segmentation format not recognized

    return mask

In [6]:
# Lists to store results for each image
results_bream = []
results_puffer = []
results_scat = []

In [7]:
# Get all image IDs
image_ids = coco.getImgIds()
print(f"Total number of images: {len(image_ids)}")

NameError: name 'coco' is not defined

In [8]:
# Process each image
for img_id in image_ids:
    # Load image information
    img_info = coco.loadImgs(img_id)[0]
    img_name_with_ext = img_info['file_name']  # e.g., 'bream_oyster_frame_000001.png'
    img_name = os.path.splitext(img_name_with_ext)[0]  # e.g., 'bream_oyster_frame_000001'
    img_height = img_info['height']
    img_width = img_info['width']

    print(f"\nProcessing image ID {img_id}, name {img_name_with_ext}")

    # Get annotation IDs for the image
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)

    if img_name_with_ext in depth_map_directories:
        # For the 3 specific images
        depth_map_directory = depth_map_directories[img_name_with_ext]
        # Get all depth maps in the directory
        depth_map_files = [f for f in os.listdir(depth_map_directory) if f.endswith('_pred04.png')]
        if not depth_map_files:
            print(f"No depth maps found in {depth_map_directory} for image {img_name_with_ext}")
            continue

        # Process each depth map in the folder
        for depth_map_file in depth_map_files:
            depth_map_path = os.path.join(depth_map_directory, depth_map_file)
            depth_map = load_and_normalize_depth_map(depth_map_path)
            if depth_map is None:
                continue  # Skip if depth map couldn't be loaded

            depth_map_folder_name = os.path.basename(depth_map_directory)
            depth_map_file_name = os.path.basename(depth_map_file)
            print(f"Processing depth map: {depth_map_file_name} from folder: {depth_map_folder_name}")

            # Process each annotation
            for ann in anns:
                annotation_id = ann['id']
                category_id = ann['category_id']
                category_name = category_mapping.get(category_id, "Unknown")
                print(f"Processing annotation ID {annotation_id}, category ID {category_id}, category name {category_name}")

                # Create mask for the annotation
                mask = create_mask_from_annotation(ann, img_height, img_width)
                if mask is None:
                    print(f"Failed to create mask for annotation {annotation_id}")
                    continue  # Skip if mask couldn't be created

                # Resize depth map if necessary
                if depth_map.shape != mask.shape:
                    depth_map_resized = cv2.resize(depth_map, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
                else:
                    depth_map_resized = depth_map

                # Apply the mask to the depth map to extract depth values
                depth_values = depth_map_resized[mask == 1]
                mean_depth = depth_values.mean() if depth_values.size > 0 else 0
                normalized_depth = mean_depth  # Depth map is already normalized between 0 and 1

                # Convert normalized depth to metric depth
                zmin = 0.4  # Minimum depth in meters 
                zmax = 20.0  # Maximum depth in meters
                value_metric = (normalized_depth * (zmax - zmin)) + zmin

                print(f"Annotation ID {annotation_id}: mean depth {mean_depth}, normalized depth {normalized_depth}, value_metric {value_metric}")

                # Prepare result dictionary
                result = {
                    'image_name': img_name_with_ext,
                    'image_id': img_id,
                    'depth_map_folder': depth_map_folder_name,
                    'depth_map_file': depth_map_file_name,
                    'annotation_id': annotation_id,
                    'category_id': category_id,
                    'category_name': category_name,
                    'normalized_depth': normalized_depth,
                    'z-min': zmin,
                    'z-max': zmax,
                    'value_metric': value_metric
                }

                # Append the result to the appropriate list
                if img_name_with_ext == 'bream_oyster_frame_000001.png':
                    results_bream.append(result)
                elif img_name_with_ext == 'puffer_oyster_frame_000001.png':
                    results_puffer.append(result)
                elif img_name_with_ext == 'scat_oyster_frame_000001.png':
                    results_scat.append(result)
    else:
        # For other images, skip or process as needed
        print(f"Skipping image {img_name_with_ext}, not one of the specified images.")
        continue
        
# After processing:
# Save results for bream oyster
if results_bream:
    results_bream_df = pd.DataFrame(results_bream)
    results_bream_df.to_csv(bream_csv_path, index=False)
    print(f"\nResults for bream oyster saved to {bream_csv_path}")
else:
    print("No results for bream oyster.")

# Save results for puffer oyster
if results_puffer:
    results_puffer_df = pd.DataFrame(results_puffer)
    results_puffer_df.to_csv(puffer_csv_path, index=False)
    print(f"\nResults for puffer oyster saved to {puffer_csv_path}")
else:
    print("No results for puffer oyster.")

# Save results for scat oyster
if results_scat:
    results_scat_df = pd.DataFrame(results_scat)
    results_scat_df.to_csv(scat_csv_path, index=False)
    print(f"\nResults for scat oyster saved to {scat_csv_path}")
else:
    print("No results for scat oyster.")

NameError: name 'image_ids' is not defined